# Spark Streaming Transform Data
read data from Kafka topic, filter and reduce and write back to other Kafka Topic

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.streaming import StreamingContext
import pyspark.sql.functions as f


import datetime
from datetime import datetime
import json


# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

In [ ]:
appName="jupyter-stream"

conf = SparkConf()

# CLUSTER MANAGER
################################################################################
# set Kubernetes Master as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type).
conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")

# CONFIGURE KUBERNETES
################################################################################
# set the namespace that will be used for running the driver and executor pods.
conf.set("spark.kubernetes.namespace","frontend")
# set the docker image from which the Worker pods are created
conf.set("spark.kubernetes.container.image", "thinkportgmbh/workshops:spark-3.3.1")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

# set service account to be used
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
# authentication for service account(required to create worker pods):
conf.set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")


# CONFIGURE SPARK
################################################################################
conf.set("spark.sql.adaptive.enabled", "False")
# set driver host. In this case the ingres service for the spark driver
# find name of the driver service with 'kubectl get services' or in the helm chart configuration
conf.set("spark.driver.host", "jupyter-spark-driver.frontend.svc.cluster.local")
# set the port, If this port is busy, spark-shell tries to bind to another port.
conf.set("spark.driver.port", "29413")
# add the postgres driver jars into session
conf.set("spark.jars", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar")
conf.set("spark.driver.extraClassPath","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar")
conf.set("spark.executor.extraClassPath","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar")
#conf.set("spark.executor.extraLibrary","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar")



# CONFIGURE S3 CONNECTOR
conf.set("spark.hadoop.fs.s3a.endpoint", "minio.minio.svc.cluster.local:9000")
conf.set("spark.hadoop.fs.s3a.access.key", "trainadm")
conf.set("spark.hadoop.fs.s3a.secret.key", "train@thinkport")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")



# conf.set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1")

# CONFIGURE WORKER (Customize based on workload)
################################################################################
# set number of worker pods
conf.set("spark.executor.instances", "1")
# set memory of each worker pod
conf.set("spark.executor.memory", "1G")
# set cpu of each worker pod
conf.set("spark.executor.cores", "2")

# SPARK SESSION
################################################################################
# and last, create the spark session and pass it the config object

spark = SparkSession\
    .builder\
    .config(conf=conf) \
    .config('spark.sql.session.timeZone', 'Europe/Berlin') \
    .appName(appName)\
    .getOrCreate()

# also get the spark context
sc=spark.sparkContext
ssc = StreamingContext(sc, 2)

# change the log level to warning, to see less output
sc.setLogLevel('WARN')

# get the configuration object to check all the configurations the session was startet with
for entry in sc.getConf().getAll():
        if entry[0] in ["spark.app.name","spark.kubernetes.namespace","spark.executor.memory","spark.executor.cores","spark.driver.host","spark.master"]:
            print(entry[0],"=",entry[1])
            


#### Read Stream from Kafka

In [ ]:
# read stream from topic
df_step_1 = (spark
      .readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka-cp-kafka.kafka.svc.cluster.local:9092")
      .option("subscribe", "twitter-table")
      .option("startingOffsets", "earliest")
      .load()
     )

df_step_1.printSchema()

In [ ]:
# schema des JSON Streams definieren
jsonSchema=StructType([
    StructField('tweet_id', StringType(), False),
    StructField('created_at', TimestampType(), False),
    StructField('tweet_message', StringType(), True),
    StructField('user_name', StringType(), True),
    StructField('user_location', StringType(), True),
    StructField('user_follower_count', IntegerType(), True),
    StructField('user_friends_count', IntegerType(), True),
    StructField('retweet_count', IntegerType(), True),
    StructField('language', StringType(), True),
    StructField('hashtags', ArrayType(StringType(), True), True)
])

df_step_2= (df_step_1
            # cast binary to string and string with json schema to json object
            .select(f.from_json(f.col("value").cast("string"),jsonSchema).alias("t"))
            # un nest via
            .select("t.*")
           )

df_step_2.printSchema()

#### For Debugging: write stream to console

In [ ]:
stream_query_debug=(df_step_2
                    .writeStream.format("console")
                    .option("truncate", "false")
                    .outputMode("append")
                    .start()
                    .awaitTermination()
                   )

In [ ]:
s3_write_avro=(df_step_2
          .writeStream
          .format("avro")
          .outputMode("append")
          .option("path", "s3a://twitter/avro")
          .option("checkpointLocation", "/opt/spark/work-dir/")
          .trigger(processingTime='10 seconds')
          .start()
          .awaitTermination()
         )

# BITTE Spark herunterfahren

In [ ]:
# Terminate Spark Session
# shut down executor pods
spark.stop()
